In [ ]:
import polars as pl
from polars.exceptions import ColumnNotFoundError
import numpy as np
import ast
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import pickle
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error

from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

from bot.analysis.pair_trading import backtest
from bot.utils.pair_trading import make_df_from_orderbooks, make_trunc_df, create_zscore_df, select_cols_1tf
from bot.analysis.strategy_analysis import analyze_strategy
from bot.utils.files import load_config

#### Подготовка данных

In [ ]:
config = load_config('./bot/config/config.yaml')

method = config['spr_method']
valid_time = config['valid_time']
end_time = config['end_time']
search_space = config['search_space']

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

token_pairs = []
with open('./data/token_pairs.txt', 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))

In [ ]:
leverage = 2

space = [('4h', np.int64(w)) for w in search_space['4h']] + [('1h', np.int64(w)) for w in search_space['1h']]
in_params = (2.5, 2.25, 2.0, 1.8, 1.6)
out_params = (0.0, 0.25, 0.5)
# out_params = (0.25, )

dist_in = 0
dist_out = 0

s_len= len(space) * len(in_params) * len(out_params) * len(token_pairs)

print('Whole search space size:', s_len)

In [ ]:
metrics_arr = []

with tqdm(total=s_len, desc="Обработка", unit="iter") as progress_bar:
    for token_1, token_2 in token_pairs:
        dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
        ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
        dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
        ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

        filepath = f'./data/pair_backtest/{token_1}_{token_2}_{method}_full.parquet'
        try:
            df = pl.read_parquet(filepath, low_memory=True, rechunk=True, use_pyarrow=True)
        except FileNotFoundError:
            print(token_1, token_2)
            continue

        end_date = df['time'][-1]

        for tf, wind in space:
            for in_ in (in_params):
                for out_ in (out_params):
                    progress_bar.update(1)
                    metrics = dict()

                    cols = ['time', 'ts', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
                            f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
                            f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
                            f'z_score_{wind}_{tf}']

                    tdf = df.select(cols).rename({f'z_score_{wind}_{tf}': 'z_score'}).drop_nulls()

                    trades_df = backtest(tdf, token_1, token_2, dp_1, dp_2, ps_1, ps_2,
                            thresh_low_in=-in_, thresh_low_out=-out_, thresh_high_in=in_, thresh_high_out=out_,
                            long_possible=True, short_possible=True,
                            balance=100, order_size=50, fee_rate=0.00055, stop_loss_std=5.0, sl_method='leave',
                            sl_seconds = 60, leverage=leverage, dist_in=0, dist_out=0,
                            verbose=0)

                    if trades_df.height > 1:
                        metrics = analyze_strategy(trades_df, start_date=valid_time, end_date=end_date, initial_balance=100.0)

                    if metrics:
                        metrics_arr.append({
                            'token_1': token_1, 'token_2': token_2, 'tf': tf, 'wind': wind,
                            'thresh_in': in_, 'thresh_out': out_, 'dist_in': dist_in, 'dist_out': dist_out,
                            'n_trades': metrics['n_trades'], 'duration_avg': metrics['duration_avg'],
                            'stop_losses': metrics['stop_losses'], 'liquidations': metrics['liquidations'],
                            'profit': metrics['profit'], 'profit_ratio': metrics['profit_ratio'],
                            'max_profit': metrics['max_profit'], 'max_drawdown': metrics['max_drawdown'],
                            'max_loss': metrics['max_loss'], 'avg_profit': metrics['avg_profit'],
                            'profit_std': metrics['profit_std']})

mass_output = pl.DataFrame(metrics_arr).sort(by='profit', descending=True)

In [ ]:
# Выбираем наилучшие параметры торговли
tf_filter = mass_output.group_by('tf', 'wind', 'thresh_in', 'thresh_out').agg(
        pl.col('profit').mean().round(2).alias('avg_profit'),
        pl.col('profit_ratio').mean().round(2).alias('avg_pr_ratio'),
        pl.col('max_loss').min().round(2).alias('max_loss'),
        pl.col('max_drawdown').mean().round(2).alias('avg_drawdown'),
        pl.col('max_drawdown').min().round(2).alias('max_drawdown'),
        pl.col('n_trades').mean().cast(pl.Int64).alias('avg_trades'),
    ).sort('avg_pr_ratio', descending=True)
tf_filter.head(10)

In [ ]:
tf_filter.filter((pl.col('tf') == '4h') & (pl.col('wind') == 30))

In [ ]:
# Результаты для конкретных пар на общем тесте
res_df = mass_output.group_by('token_1', 'token_2').agg(
            pl.col('profit').mean().round(2).alias('avg_profit'),
            pl.col('profit').std().round(2).alias('profit_std'),
            pl.col('max_loss').min().round(2).alias('max_loss'),
            pl.col('max_drawdown').mean().round(2).alias('avg_drawdown'),
            pl.col('max_drawdown').min().round(2).alias('max_drawdown'),
            pl.col('duration_avg').mean().alias('avg_duration'),
            pl.col('profit_ratio').mean().round(2).alias('avg_pr_ratio'),
            pl.col('n_trades').mean().cast(pl.Int64).alias('avg_trades'),
        ).sort('avg_pr_ratio', descending=True)

In [ ]:
# Выкидываем все пары, показашие плохой результат, и сохраняем хорошие пары в файл
res_df = res_df.filter(
    (pl.col('avg_profit') > 1.0) & (pl.col('avg_pr_ratio') > 0.5)
)
res_df

In [ ]:
# res_df.write_parquet(f'./data/pair_selection/common_params_{method}_result.parquet')

In [ ]:
# res_df.filter(pl.col('token_1') == 'ARKM')

In [ ]:
# Теперь для плюсовых пар ещё раз прогоним общий бектест для подбора наилучших параметров только на этих парах.
token_pairs = []
for row in res_df.iter_rows(named=True):
    token_pairs.append([row['token_1'], row['token_2']])
len(token_pairs)

In [ ]:
# Оставляем только лучшие параметры
leverage = 2
tf = '1h'
wind = 72
in_ = 2.25
out_ = 0.25
dist_in = 0
dist_out = 0

metrics_arr = []

for token_1, token_2 in tqdm(token_pairs):
    dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
    ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
    dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
    ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

    filepath = f'./data/pair_backtest/{token_1}_{token_2}_{method}_full.parquet'
    try:
        df = pl.read_parquet(filepath, low_memory=True, rechunk=True, use_pyarrow=True)
    except FileNotFoundError:
        continue

    end_date = df['time'][-1]
    metrics = dict()

    cols = ['time', 'ts', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
            f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
            f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
            f'z_score_{wind}_{tf}']

    tdf = df.select(cols).rename({f'z_score_{wind}_{tf}': 'z_score'}).drop_nulls()

    trades_df = backtest(tdf, token_1, token_2, dp_1, dp_2, ps_1, ps_2,
            thresh_low_in=-in_, thresh_low_out=-out_, thresh_high_in=in_, thresh_high_out=out_,
            long_possible=True, short_possible=True,
            balance=100, order_size=50, fee_rate=0.00055, stop_loss_std=5.0, sl_method='leave',
            sl_seconds = 60, leverage=leverage, dist_in=0, dist_out=0,
            verbose=0)

    if trades_df.height > 1:
        metrics = analyze_strategy(trades_df, start_date=valid_time, end_date=end_date, initial_balance=100.0)

    if metrics:
        metrics_arr.append({
            'token_1': token_1, 'token_2': token_2, 'tf': tf, 'wind': wind,
            'thresh_in': in_, 'thresh_out': out_, 'dist_in': dist_in, 'dist_out': dist_out,
            'n_trades': metrics['n_trades'], 'duration_avg': metrics['duration_avg'],
            'stop_losses': metrics['stop_losses'], 'liquidations': metrics['liquidations'],
            'profit': metrics['profit'], 'profit_ratio': metrics['profit_ratio'],
            'max_profit': metrics['max_profit'], 'max_drawdown': metrics['max_drawdown'],
            'max_loss': metrics['max_loss'], 'avg_profit': metrics['avg_profit'],
            'profit_std': metrics['profit_std']})

best_output = pl.DataFrame(metrics_arr).sort(by='profit', descending=True)

In [ ]:
best_output['profit'].sum()

In [ ]:
best_output.height

In [ ]:
best_res = best_output.group_by('token_1', 'token_2').agg(
            pl.col('profit').sum().round(2).alias('profit'),
            pl.col('max_loss').min().round(2).alias('max_loss'),
            pl.col('max_drawdown').mean().round(2).alias('avg_drawdown'),
            pl.col('max_drawdown').min().round(2).alias('max_drawdown'),
            pl.col('duration_avg').mean().alias('duration_avg'),
            pl.col('profit_ratio').mean().round(2).alias('profit_ratio'),
            pl.col('n_trades').mean().cast(pl.Int64).alias('n_trades'),
        ).filter(
            pl.col('profit') > 2
        ).sort('profit_ratio', descending=True)
best_res

In [ ]:
best_res.select(
    'token_1', 'token_2', 'profit', 'profit_ratio', 'max_loss', 'max_drawdown', 'duration_avg', 'n_trades'
).write_parquet(f'./data/pair_selection/best_params_{method}_result.parquet')

#### Отбор монет

In [ ]:
method = 'dist'
pairs = pl.read_parquet('./data/pair_selection/curr_pairs.parquet')

common_params = pl.read_parquet(f'./data/pair_selection/common_params_{method}_result.parquet')
common_params = common_params.rename({'token_1': 'coin1', 'token_2': 'coin2', 'avg_profit': 'avg_pr_all', 'profit_std': 'pr_std_all',
                                     'max_loss': 'loss_all', 'avg_drawdown': 'avg_drdn_all', 'max_drawdown': 'max_drdn_all',
                                     'avg_duration': 'dur_all', 'avg_pr_ratio': 'pr_rat_all', 'avg_trades': 'trades_all'})

best_params = pl.read_parquet(f'./data/pair_selection/best_params_{method}_result.parquet')
best_params = best_params.rename({'token_1': 'coin1', 'token_2': 'coin2', 'profit': 'pr_best', 'profit_ratio': 'pr_rat_best',
                                     'max_loss': 'loss_best', 'max_drawdown': 'max_drdn_best',
                                     'duration_avg': 'dur_best', 'n_trades': 'trades_best'})

pairs = pairs.with_columns(
    pl.col('spr_dist').round(2),
    pl.col('spr_std').round(2),
    pl.col('corr').round(2),
    pl.col('pv_1').round(2),
    pl.col('pv_2').round(2),
)

pairs = pairs.join(best_params, on=['coin1', 'coin2'])

cols_to_rename = {'pr_best': 'profit', 'pr_rat_best': 'profit_ratio', 'loss_best': 'max_loss', 'max_drdn_best': 'max_drn',
                 'dur_best': 'avg_duration', 'trades_best': 'trades'}
pairs = pairs.rename(cols_to_rename).sort('profit_ratio', descending=True)

pairs = pairs.filter(
    (abs(pl.col('beta_1') - pl.col('beta_2')) < 1.4) & (pl.col('profit') > 2) & (pl.col('spr_dist') < 2.8) & (pl.col('corr') > 0.75)
    & (pl.col('pv_1') < 0.05) & (pl.col('pv_2') < 0.05)
).sort(by='profit', descending=True)

In [ ]:
pairs#[0:30]

In [ ]:
# Жадный метод
used_tokens = []
trade_pairs_list = []
total_profit = 0

for row in pairs.iter_rows(named=True):
    t1 = row['coin1']
    t2 = row['coin2']
    profit = row['profit']
    
    if t1 in used_tokens or t2 in used_tokens:
        continue

    trade_pairs_list.append((t1, t2))
    used_tokens.append(t1)
    used_tokens.append(t2)
    total_profit += profit
    print(t1, t2, profit)
print('Общий профит:', round(total_profit, 1))

In [ ]:
import networkx as nx

In [ ]:
df = pairs.filter(
    abs(pl.col('beta_1') - pl.col('beta_2')) < 1.8
).with_columns([
        pl.when(pl.col("coin1") < pl.col("coin2")).then(pl.col("coin1")).otherwise(pl.col("coin2")).alias("a"),
        pl.when(pl.col("coin1") < pl.col("coin2")).then(pl.col("coin2")).otherwise(pl.col("coin1")).alias("b"),
])

In [ ]:
edges = df.group_by(["a","b"]).agg(
    pl.max("profit").alias("weight")
)

In [ ]:
G = nx.Graph()
for row in edges.to_dicts():
    # row: {"a": "BLUR", "b":"MANTA", "weight": 28.33}
    G.add_edge(row["a"], row["b"], weight=float(row["weight"]))

In [ ]:
matching = nx.max_weight_matching(G, maxcardinality=False, weight="weight")

In [ ]:
matched_rows = []
total = 0.0
for u, v in matching:
    w = G[u][v]["weight"]
    matched_rows.append({"coin1": u, "coin2": v, "profit": w})
    total += w

In [ ]:
matched_df = pl.DataFrame(matched_rows)

In [ ]:
print("Matched pairs:")
print(matched_df[:])
print(f"Total profit: {total:.2f}")

In [ ]:
pairs.filter(pl.col('coin1') == 'DYDX')

In [ ]:
token_pairs = []
with open('./bot/config/token_pairs.txt', 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))

In [ ]:
tf = '1h'
wind = 72
winds = np.array([wind])

valid_length = 5
train_length = 3
min_order = 42

end_time = datetime.now().replace(tzinfo=ZoneInfo("Europe/Moscow"))
valid_time = (end_time - timedelta(days=valid_length)).replace(hour=0, minute=0, second=0, microsecond=0)
start_time = valid_time - timedelta(days=train_length)
start_ts = int(datetime.timestamp(valid_time))


# for row in pairs.iter_rows(named=True):
#     token_1 = row['coin1']
#     token_2 = row['coin2']
for token_1, token_2 in token_pairs:
    df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                     start_time=start_time,
                                     end_time=end_time)
    df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                         start_time=start_time,
                                         end_time=end_time)    
    df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)
    tick_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=start_time)
    agg_df = make_trunc_df(df, timeframe=tf, token_1=token_1, token_2=token_2, method='triple')
    spread_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, winds, min_order, start_ts, 
                                 median_length=6, spr_method='dist')
    plt.figure(figsize=(15, 3))
    plt.title(f'{token_1} - {token_2} (tf: {tf}; wind: {wind})')
    plt.plot(spread_df['time'], spread_df[f'z_score_{wind}_{tf}'], c='green')
    plt.grid()
    plt.show()


In [ ]:
# Плохие пары: SUSHI - SEI; CHZ - VET; MANTA - CELO; 

# Хорошие пары: ARKM - OP; ARKM - CELO; IOTA - MANA; GMT - BLUR; XRP - LDO; ARKM - STX; ARKM - ONDO; IOTA - KAS; 

# Непонятные: ARKM - MANTA; 

In [ ]:
trading_history = db_manager.get_table('trading_history', df_type='polars')

res_df = trading_history.group_by('token_1', 'token_2').agg(
        pl.col('profit').sum().round(2),
        pl.col('profit').min().round(2).alias('min_profit'),
        pl.col('profit').max().round(2).alias('max_profit'),
        (pl.col('profit').sum() / pl.col('profit').len()).round(2).alias('avg_profit'),
        pl.col('profit').len().alias('n_trades'),
    
    ).sort(by='profit', descending=True)

res_df = res_df.join(pairs, left_on=('token_1', 'token_2'), right_on=('coin1', 'coin2')).sort('profit', descending=True)
# res_df = res_df.rename({'dist': 'spr_dist', 'std': 'spr_std'})
res_df.shape

In [ ]:
res_df

In [ ]:
cols = ['token_1', 'token_2', 'profit', 'min_profit', 'max_profit', 'avg_profit', 'n_trades', 'spr_dist', 'spr_std', 'std_1', 'std_2',
       'dist_1', 'dist_2', 'corr', 'pv_1', 'pv_2', 'avg_pr_all', 'pr_std_all', 'loss_all', 'avg_drdn_all', 'max_drdn_all', 'pr_rat_all',
       'pr_best', 'pr_rat_best', 'loss_best', 'max_drdn_best']

In [ ]:
res_df.select(cols).head(10)

In [ ]:
res_df.select(cols).tail(5)

In [ ]:
pairs.filter((pl.col('coin1') == 'CELO') & (pl.col('coin2') == 'IMX'))

#### ML

In [ ]:
df = pairs.select('coin1', 'coin2', 'dist', 'std', 'corr', 'pv_1', 'pv_2', 'pr_ind', 'pr_rat_ind', 'avg_pr_all', 'pr_rat_all',
            'pr_best', 'pr_rat_best', 'loss_best', 'max_drdn_best', 'dur_best', 'trades_best', 
            # 'pr_test', 'pr_rat_test', 'loss_test', 'max_drdn_test', 'dur_test', 'trades_test'
                 )

In [ ]:
df.shape

In [ ]:
df.sample(3)

In [ ]:
X, y = df.drop('pr_rat_test'), df['pr_rat_test']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
X_test.with_columns(
    pl.Series(y_test).alias('pr_rat_real')
).filter((pl.col('pr_rat_best') > 0.5) & (pl.col('pr_rat_all') > 0.5))#['pr_rat_real'].sum()

In [ ]:
4.753

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train);
lr_preds = lr.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), lr_preds)

In [ ]:
# 0.8532 -> 0.8286 (добавил фичу std)

In [ ]:
for name, coef in zip(lr.feature_names_in_, lr.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
params = {'eps': [0.0001, 0.001, 0.01]}

lasso = LassoCV(random_state=42)
gcv = GridSearchCV(lasso, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
lasso = LassoCV(random_state=42, eps=0.0001)
lasso.fit(X_train, y_train);
lasso_preds = lasso.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), lasso_preds)

In [ ]:
# 0.8211

In [ ]:
for name, coef in zip(lasso.feature_names_in_, lasso.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
ridge = RidgeCV()
ridge.fit(X_train, y_train);
ridge_preds = ridge.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), ridge_preds)

In [ ]:
# 0.8118 -> 0.8117 (+ std)

In [ ]:
for name, coef in zip(ridge.feature_names_in_, ridge.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
params = {'max_features': [4, 6, 8, 10, 13], 'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5, 6, 8, 10, 12, 15],
          'n_estimators': [40, 60, 80, 100, 125, 150]}

rfr = RandomForestRegressor(random_state=42)
gcv = GridSearchCV(rfr, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
rf = RandomForestRegressor(random_state=42, n_estimators=80, max_depth=12, max_features=6, min_samples_leaf=5)
rf.fit(X_train, y_train);
rf_preds = rf.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), rf_preds)

In [ ]:
# 0.7634 -> 0.7750 (+std)

In [ ]:
for name, coef in zip(rf.feature_names_in_, rf.feature_importances_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
X_test.with_columns(
    pl.Series(rf_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)#['pr_rat_real'].sum()

In [ ]:
# 4.325

In [ ]:
params = {'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2], 'max_features': [4, 6, 8, 10, 13], 
          'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5, 6, 8, 10, 12, 15],
          'n_estimators': [40, 60, 80, 100, 125, 150]}

gbr = GradientBoostingRegressor(random_state=42)
gcv = GridSearchCV(gbr, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
gbr = GradientBoostingRegressor(random_state=42, learning_rate=0.01, n_estimators=80, max_depth=5, 
                                max_features=4, min_samples_leaf=7)
gbr.fit(X_train, y_train);
gbr_preds = gbr.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), gbr_preds)

In [ ]:
# 0.8004 -> 0.8014

In [ ]:
import catboost as cb

In [ ]:
params = {'learning_rate': [0.01, 0.03, 0.1, 0.3], 'iterations': [250, 500, 1000], 
          'depth': [6, 8, 10],
          'l2_leaf_reg': [1, 3, 5, 7]}
cbr = cb.CatBoostRegressor(random_state=42, verbose=False)
gcv = GridSearchCV(cbr, params, cv=5, verbose=0)
gcv.fit(X_train.to_numpy(), y_train.to_numpy());

In [ ]:
gcv.best_params_

In [ ]:
train_pool = cb.Pool(X_train.to_numpy(), y_train.to_numpy())

In [ ]:
param_grid = {
    'iterations': tune.randint(100, 1500),
    'learning_rate': tune.loguniform(1e-3, 0.5),
    'depth': tune.randint(4, 12),
    'l2_leaf_reg': tune.loguniform(1, 10),
}

In [ ]:
cb.__version__

In [ ]:
from catboost.utils import grid_search

In [ ]:
cbr = cb.CatBoostRegressor(random_state=42, verbose=False, learning_rate=0.01, iterations=800, depth=8,
                          loss_function='Expectile:alpha=0.7'
                          )
cbr.fit(X_train.to_numpy(), y_train.to_numpy());
cbr_preds = cbr.predict(X_test.to_numpy())
root_mean_squared_error(y_test.to_numpy(), cbr_preds)

In [ ]:
cbr.save_model('./data/catboost_model.json', format='json')

In [ ]:
# 0.7172

In [ ]:
for name, coef in zip(X_test.columns, cbr.feature_importances_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
X_test.with_columns(
    pl.Series(cbr_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)['pr_rat_real'].sum()

In [ ]:
# RMSE: 5.73
# Quantile, alpha=0.7 : 9.45,  RMSE: 0.7362
# Quantile, alpha=0.65: 9.07,  RMSE: 0.7172
# Quantile, alpha=0.6 : 10.04, RMSE: 0.7522
# Quantile, alpha=0.55: 6.65,  RMSE: 0.7392
# Quantile, alpha=0.5 : 5.06, RMSE: 0.7650
# Quantile, alpha=0.4 : 3.56, RMSE: 0.7893
# Quantile, alpha=0.3 : 3.56,  RMSE: 0.8197

# Expectile, alpha=0.3 : 6.25, RMSE: 0.7571
# Expectile, alpha=0.4 : 8.97, RMSE: 0.7603
# Expectile, alpha=0.5 : 9.60, RMSE: 0.7435
# Expectile, alpha=0.6 : 8.97, RMSE: 0.7535
# Expectile, alpha=0.65: 9.60, RMSE: 0.7323
# Expectile, alpha=0.7 : 9.60, RMSE: 0.7214
# Expectile, alpha=0.75: 10.04, RMSE: 0.7260
# Expectile, alpha=0.8 : 9.87, RMSE: 0.7344

In [ ]:
X_test.with_columns(
    pl.Series(cbr_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('./data/catboost_model.json', format='json')

In [ ]:
df.sample(2)

In [ ]:
df = df.drop('coin1', 'coin2', 'dur_best', 
            # 'dur_test', 'trades_test', 'loss_test', 'max_drdn_test', 'pr_test'
            )

X = df.to_numpy()

In [ ]:
preds = model.predict(X)

In [ ]:
df = pairs.select('coin1', 'coin2', 'dist', 'std', 'corr', 'pv_1', 'pv_2', 'pr_ind', 'pr_rat_ind', 'avg_pr_all', 'pr_rat_all',
            'pr_best', 'pr_rat_best', 'loss_best', 'max_drdn_best', 'dur_best',
            # 'pr_test', 'pr_rat_test', 'loss_test', 'max_drdn_test', 'dur_test', 'trades_test'
                 )

trade_pairs_df = df.with_columns(
        pl.Series(preds).alias('pred'),
    ).filter(
        (pl.col('pr_rat_ind') > 0.4) & (pl.col('pr_rat_all') > 0.4) & (pl.col('pr_rat_best') > 0.4) #& (pl.col('pred') > 0.4)
    ).sort(by='pr_rat_all', descending=True)


In [ ]:
trade_pairs_df

In [ ]:
used_tokens = []
trade_pairs_list = []

for row in trade_pairs_df.iter_rows(named=True):
    t1 = row['coin1']
    t2 = row['coin2']
    
    if t1 in used_tokens or t2 in used_tokens:
        continue

    trade_pairs_list.append((t1, t2))
    used_tokens.append(t1)
    used_tokens.append(t2)

In [ ]:
used_tokens

In [ ]:
trade_pairs_list

In [ ]:
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [ ]:
current_pairs = db_manager.get_table('pairs', df_type='polars')
current_pairs

In [ ]:
for row in current_pairs.iter_rows(named=True):
    t1 = row['token_1'][:-5]
    t2 = row['token_2'][:-5]
    
    if (t1, t2) not in trade_pairs_list:
        trade_pairs_list.append((t1, t2))
        print((t1, t2))

In [ ]:
with open('./jaref_bot/config/token_pairs.txt', 'w') as file:
    for pair in trade_pairs_list:
        file.write(f"{pair[0]} {pair[1]}\n")